# RL Exercise 6 - Training with Ray and Serving with Clipper

**GOAL:** The goal of this exercise is to show how to train a policy with Ray and to deploy it with Clipper in a fun, interactive way.

We will train an agent to play Pong, and then we will play Pong against the policy that we trained.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
import pong_py
import ray

from ray.rllib.ppo import PPOAgent, DEFAULT_CONFIG

Start Ray.

In [ ]:
ray.init(num_workers=0)

The cell below is a hack. The explanation is as follows. Internally within the `PPOAgent` constructor, a number of actors are created, and these actors will instantiate gym environments using the command `gym.make('PongJS-v0')`. The command `gym.make` knows how to instantiate a number of pre-defined environments that are shipped with the `gym` module. However, the `PongJS-v0` environment is defined in the `pong_py` module and is registered with the `gym` module when the `import pong_py` statement gets run.

Therefore, for the actors to successfully instantiate the gym environments, the `pong_py` module must be imported on the actors. This is why we define a remote function `import_pong_py` which closes over the `pong_py` environment. When the actors are created, that remote function is unpickled on the actors which forces the `pong_py` module to be imported, which enables the `gym` module to create the `PongJS-v0` environment.

In [ ]:
# This is a hack.
@ray.remote
def import_pong_py():
    pong_py

Instantiate an agent that can be trained using Proximal Policy Optimization (PPO).

In [ ]:
config = DEFAULT_CONFIG.copy()
config['num_workers'] = 3
config['num_sgd_iter'] = 20
config['sgd_batchsize'] = 8196
config['model']['fcnet_hiddens'] = [32, 32]

agent = PPOAgent('PongJS-v0', config)

Train the `PPOAgent` for some number of iterations.

**EXERCISE:** You will need to experiment with the number of iterations as well as with the configuration to get the agent to learn something reasonable.

In [ ]:
for i in range(2):
    result = agent.train()

Use the agent manually by calling `agent.compute_action` and see the rewards you get are consistent with the rewards printed during the training procedure.

In [ ]:
env = gym.make('PongJS-v0')

for _ in range(20):
    state = env.reset()
    done = False
    cumulative_reward = 0

    while not done:
        action = agent.compute_action(state)
        state, reward, done, _ = env.step(action)
        cumulative_reward += reward

    print(cumulative_reward)

Checkpoint the agent so that the relevant model can be saved and deployed to Clipper.

In [ ]:
checkpoint_path = agent.save()

## Play Against the Policy

In this section, we will play Pong against the policy that we just trained. The game will be played in your browser, and the policy that we trained will be served by Clipper.

**EXERCISE:** Deploy your policy using Clipper. Follow the instructions that get printed below to play Pong against the deployed policy. You'll need to deploy all of the data that is saved in the directory `os.path.dirname(checkpoint_path)`.

In [ ]:
raise NotImplementedError